In [ ]:
#Hidden
from datascience import Table
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('fivethirtyeight')

import pandas as pd

## Just for reference, since datasience is evolving rapidly
import datascience as ds
ds.__version__

# Hypothesis Testing and Thinking about Causation

As we begin to speak about hypothesis, one question that comes up concerns causation. Today, we will work a bit with hypothesis testing and then we will begin to consider causation. We will use out elderly poor "sample" for hypothesis testing, and we will look the relationship between life expectancy and wealth in a similar "sample." 

/I/ Permutation test

/II/ A-B test

/III/ Bivariate relationships

## /I/ Permutation Test and the Poverty Rate

Last time, we calculated a 95% CI for poverty rates among those identified as black and white in Census Bureau statistics. The procedure qualifies as one traditional way to test a hypothesis. This time, we want to do a permutation test on the small data-set we looked at last time, namely poverty rates between elderly people identified as belonging to different races

First, take a look at what the data looks like.

In [ ]:
#Hidden
pov_long = Table.read_table("us65pov_long.csv")
pov_long

Recall from last time that we have population level data, BUT that we are treating it as a random sample for 36 consecutive years. Recall also that we remarked that stark differences appeared to exist in poverty rates, as suggested by several visualizations.

In [ ]:
#How many years are we calculating?
pov_long.where(pov_long["category"],"all").num_rows

In [ ]:
#Histogram of poverty rate of white people, several options
pov_long.select("65pc").where(pov_long["category"], "white").hist(bins=np.arange(0, 60, 1))
plt.xlabel("poverty rate of white")

In [ ]:
#Histogram of poverty rate of black people
pov_long.select("65pc").where(pov_long["category"], "black").hist(bins=36, normed=True)
plt.xlabel("poverty rate of black")

The two histograms suggested there is a difference between poverty rates. Is there really?

Let us do a permutation test to help us with our hypothesis, and recall that we need several components in a hypothesis test:

**Null Hypothesis**

**Alternate Hypothesis**

**Test Statistic**

**P-Value**

First, assume we have the null hypothesis: 
Between the 1960s and 2000s, there was no difference between the poverty rates (65pc) typical for elderly white and black "samples" in the US.

Second, the alternative hypothesis:
There is a difference between the typical poverty rates (65pc) of white and black elderly "samples".

In [ ]:
#Permutation test for poverty rate between white and black people

meanW = np.mean(pov_long.where(pov_long["category"], "white")["65pc"])
meanB = np.mean(pov_long.where(pov_long["category"], "black")["65pc"])
obs_diff = meanB - meanW

repetitions=10000
diffs = []

for i in range(repetitions):
    
    # sample WITHOUT replacement, same number as original sample size
    resample = pov_long.sample()
    
    # Compute the difference of the means of the resampled values, between elderly black and white
    dd = np.mean(resample.where(pov_long["category"], "black")["65pc"]) \
    - np.mean(resample.where(pov_long["category"], "white")["65pc"])
    diffs.append([dd])

# Display results   
diffs = Table([diffs],['diff_in_means'])
diffs.hist(bins=20, normed=True)
plots.xlabel('Approx null distribution of difference in means')
plots.title('Permutation Test')
print('Observed difference in means: ', obs_diff)

The figure shows that under the null hypothesis of equal distributions in the "samples," the empirical distribution of the difference between the sample means of the two groups is roughly bell shaped, centered at 0, stretching from about −8 percent to 8 percent. 

The **observed difference** in the original sample is about 21.4 percent

### Your turn

Write out a conclusion; would you incorporate a 95% CI? 

## /II/ AB Test and the Poverty Rate

Now, let us analyze the poverty rate between the white elderly "sample" and non-white elderly (including those identified as black, Asian and Hispanic). We will be using AB Testing.

First, do a little modification to the table where in category, white is 1, non-white is 0.

In [ ]:
#Since we only need white and non-white category, remove "all"
pov_long_new = pov_long.where(pov_long["category"]!="all")
new_races = []
for i in pov_long_new["category"]:
    if (i == "white"):
        new_races.append(1)
    else:
        new_races.append(0)
pov_long_new.append_column("whiteIndicator", new_races)
pov_long_new

Let us define a function _bootstrap_ab_test_ that performs A-B testing using the bootstrap method and the difference in sample means as the test statistic. The null hypothesis is that the two underlying distributions in the population are equal; the alternative is that they are not.

The arguments of the function are:
•	the name of the table that contains the data in the original sample

•	the label of the column containing the response variable (that is, the variable whose distribution is of interest)

•	the label of the column containing the code 0 for Category A and 1 for Category B

•	the number of repetitions of the resampling procedure

The function returns the observed difference in means, the bootstrap empirical distribution of the difference in means, and the bootstrap empirical P-value. Because the alternative simply says that the two underlying distributions are different, the P-value is computed as the proportion of sampled differences that are at least as large in absolute value as the absolute value of the observed difference.

In [ ]:
"""Bootstrap A/B test for the difference in the mean response
Assumes A=0, B=1"""

def bootstrap_AB_test(samp_table, response_label, ab_label, repetitions):
    
    # Sort the sample table according to the A/B column; 
    # then select only the column of effects.
    response = samp_table.sort(ab_label).select(response_label)
    
    # Find the number of entries in Category A.
    n_A = samp_table.where(samp_table[ab_label],0).num_rows
      
    # Calculate the observed value of the test statistic.
    meanA = np.mean(response[response_label][:n_A])
    meanB = np.mean(response[response_label][n_A:])
    obs_diff = meanA - meanB
    
    # Run the bootstrap procedure and get a list of resampled differences in means
    diffs = []
    for i in range(repetitions):
        resample = response.sample(with_replacement=True)
        d = np.mean(resample[response_label][:n_A]) - np.mean(resample[response_label][n_A:])
        diffs.append([d])
    
    # Compute the bootstrap empirical P-value
    diff_array = np.array(diffs)
    p_value = np.count_nonzero(abs(diff_array) >= abs(obs_diff))/repetitions
    
    # Display results
    diffs = Table([diffs],['diff_in_means'])
    diffs.hist(bins=20,normed=True)
    plots.xlabel('Approx null distribution of difference in means')
    plots.title('Bootstrap A-B Test')
    print("Observed difference in means: ", obs_diff)
    print("Bootstrap empirical P-value: ", p_value)

We can now use the function bootstrap_AB_test to compare the poverty rate of white and non-white. 

### Your turn: 

From the result, we can we reject the null hypothesis? What conclusion can we come to about that poverty rate of white and non-white elderly "samples"?

# /III/ More than a Correlation? 

Raj Chetty provides compelling evidence that wealth and health are strongly associated, and the _JAMA_ paper stops short of invoking the "Big-C": causation. 

Next week, we will talk in the main class about interpreting regression in more detail, and from our reading today there are a few things to note:

--Chetty and his co-authors confirm that there seems to be a very strong association between health and wealth
(see the powerful visaualization at Gap Minder, Global Trends Wealth & Health of Nations)

--The two-variable linear regression, very briefly: 

•	A two-variable linear regression relates two variables by fitting a straight line to the data

•	The model allows us to:

/i/ quantify a theory about the relationship between two variables;

/ii/ measure the statistical significance of that relationship; and

/iii/ predict the value of the dependent variable for a given value of the independent variable.

•	Note: a “linear” model is an assumption that may not best represent the actual relationship

--Deaton suggests that we need to be wary about over-interpreting this association; and here are some common biases:

•	**Omitted Variables Bias** – the relationship between two variables may be (at least partially) a result of a third variable that is related to both (and that is currently inside the error term)
•	**Selection Bias** – the process of selecting the sample or observations may select a non-random and non-representative sample
•	**Reverse Causality** – rather than X causing Y, Y may have a causal influence on X

We will import some more software, and Ryan Edwards posed similar questions recently using data from the US.

In [ ]:
import statsmodels.api as sm
import pandas as pd

In [ ]:
usagdple= Table.read_table ("usagdple.csv")
usagdple

Let's start with some visualizations.
Create scatter plots of:

--life expectancy at birth for both sexes, e0b, versus year

--log real GDP per capita, logrgdppc, versus year

--life expectancy at birth for both sexes, e0b, versus log real GDP per capita, logrgdppc 

In [ ]:
usagdple.scatter('year','logrgdppc', fit_line=True, s=40)

### Your turn: scatter plots of key variables

In [ ]:
# Create a scatter of year vs. logrgdppc, and describe the trend

In [ ]:
# Create a scatter of logrgdppc vs. e0b, and describe the trend; does it look like anything you saw before?

In [ ]:
#This code gets us to use Pandas

usa = usagdple.to_df()
type(usa)

In [ ]:
# Our x variable is log real GDP per capita, while 'ones' is the so-called constant term /we'll talk more next week!/
x = usa[['ones','logrgdppc']]
y = usa['e0b']
multiple_regress = sm.OLS(y, x).fit()
multiple_regress.summary()

Let's discuss the results -- what looks familiar?